In [1]:
%load_ext autoreload
%autoreload 2

# IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [2]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d')

,change,changeOverTime,changePercent,close,high,label,low,open,uClose,uHigh,uLow,uOpen,uVolume,volume
date,,,,,,,,,,,,,,
2019-09-30,0.00,0.000000,0.0000,232.57,233.01,Sep 30,230.73,222.40,228.30,229.41,221.60,227.30,27560717,27356318
2019-10-01,0.64,0.002815,0.2826,226.01,233.05,Oct 1,229.60,232.19,232.83,237.84,228.60,230.65,37151742,37308230
2019-10-02,-5.75,-0.022611,-2.5684,227.78,229.09,Oct 2,228.07,232.78,229.49,233.52,218.58,226.00,37445848,36370828
2019-10-03,1.94,-0.014334,0.8574,231.74,225.67,Oct 3,217.92,221.74,231.50,225.57,224.30,222.75,31557655,30834853
2019-10-04,6.46,0.014193,2.8351,230.52,234.07,Oct 4,225.19,228.58,237.81,237.88,226.40,227.52,35754280,34896043


## Fetche SPY stocks

In [51]:
import pandas as pd
spy = [x for x in pd.read_html('https://etfdailynews.com/etf/spy/', attrs={'id': 'etfs-that-own'})[0].Symbol.values.tolist() if isinstance(x, str)]

In [ ]:
iex.init(key.test_token, api='sandbox')
# iex.init(key.token, api='cloud')
for k in spy:
    # todo read csv and add needed range
    iex.Stock(k).chart('1y').to_csv("./daily/{}.csv".format(k))